In [1]:
# change notes 11/22/24:

# moved data.yaml file to dataset folder per YOLO documentation
# dataset needs to be in same folder as notebook
# updated yaml.data to 10 classes, due to dataset error for only subset to 7 or 8
# added label dir splitting process
# added markdowns
# prevented raw_images and raw_labels from being cleared
# trained model from labels

WIP: 
<br> fix yaml location pull (yaml needs to be in same folder as images)
<br>any further fine tuning on hyperparams
<br>add validation plots,
<br>initial and predicted image subplots *from custom dataset only*
<br>live video feed,
<br>dashboard

# Import Libraries

In [1]:
# import modules
import os # file 
import shutil
import cv2 # opencv for images
import random
import numpy as np
import pandas as pd
import torch
from ultralytics import YOLO #for obeject detection
import albumentations as A # for image augmentation
from albumentations.pytorch import ToTensorV2 # for image formating
# from tqdm import tqdm  # to show processing progress
# Suppress all warnings
import warnings
warnings.filterwarnings("ignore")

# Clear the "train", "test", "valid" folders from base dir

In [2]:
# create direcories to organize images and cleanup for a new to avoid duplicate images 
def reset_directories(root_dir, directories):
    """
    Check if the specified directories exist. If they do, delete them and recreate them.
    Ensures the directories are clean before use.

    Parameters:
        directories (list): List of directories to reset.
    """
    for path in directories:
        dir_path = root_dir+path
        if os.path.exists(dir_path):
            # delete the directory and all its contents
            try:
                shutil.rmtree(dir_path)
                print(f"Deleted existing directory: {dir_path}")
            except Exception as e:
                print(f"Failed to delete {dir_path}. Reason: {e}")
        
        # Recreate the directory
        try:
            os.makedirs(dir_path, exist_ok=True)
            print(f"Recreated directory: {dir_path}")
        except Exception as e:
            print(f"Failed to create directory {dir_path}. Reason: {e}")
# base dir
dataset_base_dir = "./datasets"

# Define directories to reset
directories_to_reset = [
    "/images/train",
    "/images/valid",
    "/images/test",
    "/labels/train",
    "/labels/valid",
    "/labels/test"
]

# Reset directories
reset_directories(dataset_base_dir, directories_to_reset)

print("Directories reset and ready for use.")

Recreated directory: ./datasets/images/train
Recreated directory: ./datasets/images/valid
Recreated directory: ./datasets/images/test
Recreated directory: ./datasets/labels/train
Recreated directory: ./datasets/labels/valid
Recreated directory: ./datasets/labels/test
Directories reset and ready for use.


# Split files from "raw_image" and "raw_label" folders
#### (Note: "./datasets/raw_images" and "./datasets/raw_labels" folders must already contain all images and all labels)

In [3]:
# Set random seed for reproducibility
random.seed(42)

# source directory containing all raw images and labels
source_image_dir = "./datasets/raw_images"
source_label_dir = "./datasets/raw_labels"

# base directory for the organized dataset
base_dir = dataset_base_dir

image_dirs = {
    "train": os.path.join(base_dir, "images/train"),
    "valid": os.path.join(base_dir, "images/valid"),
    "test": os.path.join(base_dir, "images/test")
}
label_dirs = {
    "train": os.path.join(base_dir, "labels/train"),
    "valid": os.path.join(base_dir, "labels/valid"),
    "test": os.path.join(base_dir, "labels/test")
}

# Ensure directories exist
for dir_path in list(image_dirs.values()) + list(label_dirs.values()):
    os.makedirs(dir_path, exist_ok=True)

# Extract the image files
image_files = [f for f in os.listdir(source_image_dir) if f.endswith(('.jpg', '.jpeg', '.png'))]
random.shuffle(image_files)

# Create DataFrame with file paths and dataset split assignments
df = pd.DataFrame({"filename": image_files})

# Dynamically split dataset into train, valid, and test
train_size = int(0.7 * len(df))  # 70% for training
valid_size = int(0.2 * len(df))  # 20% for validation
test_size = len(df) - train_size - valid_size  # remaining 10% for testing

train_labels = ["train"] * train_size
valid_labels = ["valid"] * valid_size
test_labels = ["test"] * test_size

# Combine and assign to DataFrame
df["split"] = train_labels + valid_labels + test_labels

# Copy files to respective folders
for _, row in df.iterrows():
    img_file = row["filename"]
    split = row["split"]
    
    # Source paths
    img_src = os.path.join(source_image_dir, img_file)
    label_src = os.path.join(source_label_dir, os.path.splitext(img_file)[0] + ".txt")
    
    # Destination paths
    img_dest = os.path.join(image_dirs[split], img_file)
    label_dest = os.path.join(label_dirs[split], os.path.splitext(img_file)[0] + ".txt")
    
    # Copy image file
    if os.path.exists(img_src):
        shutil.copy(img_src, img_dest)
    
    # Copy corresponding label file
    if os.path.exists(label_src):
        shutil.copy(label_src, label_dest)

# Training

In [ ]:
model = YOLO('yolov8n.pt')
model.train(data='data.yaml', epochs = 10, val = True)

Ultralytics 8.3.36  Python-3.12.7 torch-2.5.1+cpu CPU (Intel Core(TM) i7-10750H 2.60GHz)
engine\trainer: task=detect, mode=train, model=yolov8n.pt, data=data.yaml, epochs=10, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train57, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=True, l

100%|███████████████████████████████████████████████████████████████████████████████| 755k/755k [00:00<00:00, 4.42MB/s]


Overriding model.yaml nc=80 with nc=10

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytic

train: Scanning C:\Users\User\Documents\GitHub\aai-501-final-project_dv\datasets\labels\test... 287 images, 5 backgroun

train: New cache created: C:\Users\User\Documents\GitHub\aai-501-final-project_dv\datasets\labels\test.cache
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))


...


      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


       3/10         0G       1.43      2.632      1.522        228        640: 100%|██████████| 18/18 [01:26<00:00,  4.
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 18/18 [01:27

                   all        571       7714      0.828     0.0902      0.184      0.103



      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


       4/10         0G      1.394      2.308      1.497        188        640:  83%|████████▎ | 15/18 [01:13<00:15,  5.

In [ ]:
# load YOLOv8 pre-trained model
model = YOLO("../models/yolov8n.pt")

# define augmentation pipeline
augmentation = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.Normalize(mean=(0, 0, 0), std=(1, 1, 1)),  # No change to pixel values
    ToTensorV2()  # convert to PyTorch tensor 
], bbox_params=A.BboxParams(format='yolo', label_fields=['labels']))

# define function to annotate and process images
def annotate_images(df, split, image_dirs, label_dirs, confidence_threshold=0.5):
    data = []
    split_df = df[df["split"] == split]
    for _, row in split_df.iterrows():
        image_file = row["filename"]
        # load the image
        image_path = os.path.join(source_image_dir, image_file)
        image = cv2.imread(image_path)

        # perform inference using YOLOv8
        results = model(image_path)

        # extract bounding boxes and labels
        bboxes = []
        labels = []
        for result in results[0].boxes:
            box = result.xywhn[0].cpu().numpy()  # Normalized x_center, y_center, width, height
            class_id = int(result.cls[0].cpu().numpy())
            confidence = float(result.conf[0].cpu().numpy())

            # filter by confidence threshold
            if confidence >= confidence_threshold:
                bboxes.append(box.tolist())
                labels.append(class_id)

                # add annotation details to the df list
                data.append({
                    "filename": image_file,
                    "split": split,
                    "class_id": class_id,
                    "confidence": confidence,
                    "x_center": box[0],
                    "y_center": box[1],
                    "width": box[2],
                    "height": box[3],
                })

        # apply augmentation
        if bboxes:  # Only augment if there are bounding boxes
            augmented = augmentation(image=image, bboxes=bboxes, labels=labels)
            image = augmented["image"]
            bboxes = augmented["bboxes"]
            labels = augmented["labels"]

        # convert to numpy format for saving 
        if isinstance(image, torch.Tensor):  # If tensor, convert to numpy for opencv
            image = image.permute(1, 2, 0).cpu().numpy() # changes the order of the tensor dimensions from (C, H, W) (Channel-Height-Width, common in PyTorch) to (H, W, C) (Height-Width-Channel, required by OpenCV and most image libraries).
            image = (image * 255).astype(np.uint8)  # Convert to uint8 for OpenCV

        # save the image to the appropriate directory
        output_image_path = os.path.join(image_dirs[split], image_file)
        cv2.imwrite(output_image_path, image)

        # set YOLO format labels
        label_file = os.path.splitext(image_file)[0] + ".txt"
        label_path = os.path.join(label_dirs[split], label_file)

        # create YOLO format label for the corresponding image
        with open(label_path, "w") as f:
            for bbox, class_id in zip(bboxes, labels):
                # Write each valid detection to the file in YOLO format
                f.write(f"{class_id} {bbox[0]:.6f} {bbox[1]:.6f} {bbox[2]:.6f} {bbox[3]:.6f}\n")

    return data
# annotate and process images for all splits
annotation_data = []
for split in ["train", "valid", "test"]:
    # append each list to annotation data
    annotation_data.extend(annotate_images(df, split, image_dirs, label_dirs, confidence_threshold=0.5))


print("Annotation completed and dataset organized into train, validation, and test directories.")

In [ ]:
annotation_df = pd.DataFrame(annotation_data)
annotation_df.head()